# Price Discovery Project Using Agents

For this project to run you will need to install langchain as well as its dependancies. You will also need to have a google api key as well as a SerpAPI key.

## Create agent for retrieval

In [180]:
# Iport the necessary libraries
import os
from langchain.tools.retriever import create_retriever_tool
from langchain.agents.load_tools import load_tools
from langchain.agents import AgentExecutor, Tool, create_structured_chat_agent, create_react_agent, create_self_ask_with_search_agent
from langchain_community.utilities import SearchApiAPIWrapper, SerpAPIWrapper
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_community.tools.google_lens import GoogleLensQueryRun
from langchain_community.utilities.google_lens import GoogleLensAPIWrapper
from langchain_community.utilities import GoogleSearchAPIWrapper

In [37]:
# Load LLM and Google API key
GOOGLE_API_KEY = ""  
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [77]:
# Load Serp API key
serp_api_key = ""
os.environ["SERPAPI_API_KEY"] = serp_api_key

In [250]:
# Create test prompt to be used with the agent
test_prompt = """
    
As an experienced product analyst adept at accurately determining product price ranges,
utilize the available tools {tools} to answer the question asked. 
Return ONLY a reasonable, accurate and concise price range for my product after analysing its current pricing
and that of a similar products. 

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found use your own knowledge
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""

test_prompt_template = PromptTemplate.from_template(test_prompt)

In [246]:
# Create tools to be used with by the agent
search = SerpAPIWrapper()
# tool_text = create_retriever_tool(text_retriever,
#                                      'retrieve from text db',
#                                      'Searches the product description for the product given the product description' )
toolz = [
    Tool(
    name="search item",
    description="Searches for product price using product description",
    func=search.run
),
    Tool(
    name="search image",
    description="Searches for product price using image url",
    func=search.run
    ),
    
     
]

In [252]:
# Instantiate agent and agent executor
serp_agent = create_react_agent(llm, toolz, test_prompt_template)
serp_agent_executor = AgentExecutor(agent=serp_agent, tools=toolz, verbose=True, handle_parsing_errors=True)

In [253]:
# Image description and URL
description = "iHaHa Fire Truck Toys for 1 2 3 4 5 6 Years Old Boys Toddler, 5 in 1 Kids Carrier Toy Birthday, Car Friction Power Toys with Light Sound"
image_url = "https://m.media-amazon.com/images/I/81nq8H2IlyL._AC_UL320_.jpg"

In [254]:
# Run the agent
serp_agent_executor.invoke({"input": f"what is the price range of this {description} "})



> Entering new AgentExecutor chain...


Action: search item
Action Input: iHaHa Fire Truck Toys for 1 2 3 4 5 6 Years Old Boys Toddler, 5 in 1 Kids Carrier Toy Birthday, Car Friction Power Toys with Light Sound ['Fire Truck Toys for 1 2 3 4 5 6 Years Old Boys Toddler, 5 in 1 Kids Carrier Toy Birthday, Car Friction Power Toys with Light Sound ; Buy Both: $00$00 ; Important ...', '【5 in 1 Construction Cartoon Car Trucks Toy】- This trucks cars toys for 1 2 3 4 5 year old boys includes a larger construction carrier truck and a small ...', "【Safe & Durable】 - This fire car truck toys for toddlers boys is made of safe high quality plastic with rounded edges, no corners, no burrs, so don't worry ...", 'iHaHa Fire Truck Toys for 1 2 3 4 5 6 Years Old Boys Toddler, 5 in 1 Kids Carrier Toy Birthday, Car Friction Power Toys with Light Sound. Item #: 73290918.', 'iHaHa Fire Truck Toys for 1 2 3 4 5 6 Years Old Boys Toddler, 5 in 1 Kids Carrier Toy Birthday, Car Friction Power Toys with Light Sound.', 'Shop iHaHa Fire Truck Toys for 1 2 

{'input': 'what is the price range of this iHaHa Fire Truck Toys for 1 2 3 4 5 6 Years Old Boys Toddler, 5 in 1 Kids Carrier Toy Birthday, Car Friction Power Toys with Light Sound ',
 'output': '$20-$30'}